In [ ]:
import gradio as gr
import asyncio
import nest_asyncio
from typing import Dict, List, Optional, Tuple, Any
import numpy as np
import os
import threading
from datetime import datetime
import tempfile
import logging
import warnings
import sys
import json
import io
import contextlib
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()



# Suppress httpcore warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
logging.getLogger("httpcore").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Apply nest_asyncio
nest_asyncio.apply()

# Core imports
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.tools import Tool

# Import LangChain's built-in tools
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import ShellTool

class SimpleAudio:
    """Simplified audio handler"""
    
    def __init__(self):
        self.has_stt = False
        self.has_tts = False
        self._check_audio()
        
    def _check_audio(self):
        """Check audio capabilities"""
        try:
            import speech_recognition as sr
            self.recognizer = sr.Recognizer()
            self.has_stt = True
            logger.info("✓ Speech recognition available")
        except:
            logger.warning("✗ Speech recognition not available")
            
        try:
            import edge_tts
            self.has_tts = True
            logger.info("✓ Text-to-speech available")
        except:
            logger.warning("✗ Text-to-speech not available")
    
    def transcribe(self, audio_data):
        """Transcribe audio"""
        if not self.has_stt or audio_data is None:
            return ""
            
        try:
            import speech_recognition as sr
            
            if isinstance(audio_data, tuple):
                sr_rate, audio_array = audio_data
                
                if len(audio_array) == 0:
                    return ""
                    
                if audio_array.dtype != np.int16:
                    audio_array = (audio_array * 32767).astype(np.int16)
                
                import io, wave
                buffer = io.BytesIO()
                with wave.open(buffer, 'wb') as wav:
                    wav.setnchannels(1)
                    wav.setsampwidth(2)
                    wav.setframerate(sr_rate)
                    wav.writeframes(audio_array.tobytes())
                
                buffer.seek(0)
                
                with sr.AudioFile(buffer) as source:
                    audio = self.recognizer.record(source)
                    return self.recognizer.recognize_google(audio)
                    
        except Exception as e:
            logger.error(f"Transcription error: {e}")
            return ""
    
    def speak(self, text):
        """Generate speech"""
        if not self.has_tts:
            return None
            
        try:
            import edge_tts
            import scipy.io.wavfile as wavfile
            
            async def generate():
                tts = edge_tts.Communicate(text, "en-US-AriaNeural")
                with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
                    await tts.save(f.name)
                    return f.name
            
            loop = asyncio.new_event_loop()
            try:
                mp3_file = loop.run_until_complete(generate())
                
                # Convert MP3 to WAV
                try:
                    from pydub import AudioSegment
                    audio = AudioSegment.from_mp3(mp3_file)
                    wav_file = mp3_file.replace('.mp3', '.wav')
                    audio.export(wav_file, format="wav")
                    sr, audio_data = wavfile.read(wav_file)
                    os.unlink(mp3_file)
                    os.unlink(wav_file)
                    return sr, audio_data
                except ImportError:
                    os.unlink(mp3_file)
                    logger.warning("pydub not installed for audio conversion")
                    return None
                    
            finally:
                loop.close()
                
        except Exception as e:
            logger.error(f"TTS error: {e}")
            return None

class EnhancedMCPDashboard:
    """Enhanced dashboard with better MCP tool handling and parameter introspection"""
    
    def __init__(self, mcp_servers: Dict, agent_configs: List[Dict], max_iterations: int = 5):
        logger.info("Initializing enhanced dashboard...")
        
        self.mcp_servers = mcp_servers
        self.agent_configs = agent_configs
        self.max_iterations = max_iterations  # Configurable iterations
        self.audio = SimpleAudio()
        self.agents = {}
        self.tools = []
        self.mcp_tools_info = {}  # Store MCP tool metadata
        self.executors = {}
        
        # MCP event loop
        self.mcp_loop = None
        self.mcp_thread = None
        self.cleanup_func = None
        
        # Initialize system
        self._initialize_system()
        
        # Register cleanup
        import atexit
        atexit.register(self._cleanup)
    
    def _initialize_system(self):
        """Initialize all components"""
        logger.info("Starting system initialization...")
        
        # Initialize tools
        self._init_tools()
        
        # Create agents
        self._create_agents()
        
        logger.info("✓ System initialization complete")
    
    def _init_tools(self):
        """Initialize all tools including MCP"""
        # Add LangChain's Python REPL tool with a wrapper to handle parameter format
        base_python_tool = PythonREPLTool()
        
        # Create a wrapper function that handles the parameter format
        def python_wrapper(input_data):
            """Wrapper that handles different input formats for Python REPL"""
            # Handle dict input from agent
            if isinstance(input_data, dict):
                # Try different possible key names
                code = input_data.get('command', input_data.get('code', input_data.get('query', str(input_data))))
            else:
                code = str(input_data)
            
            logger.info(f"Python REPL executing: {code[:100]}...")
            
            # Call the actual tool
            result = base_python_tool.run(code)
            
            logger.info(f"Python REPL result: {result[:200]}...")
            return result
        
        # Create the wrapped tool
        python_tool = Tool(
            name="python_repl",
            description="Execute Python code in a REPL environment. Input should be valid Python code. Returns the output of the code execution.",
            func=python_wrapper
        )
        self.tools.append(python_tool)
        
        # Add Shell/Bash tool
        shell_tool = ShellTool()
        shell_tool.name = "bash_command"
        shell_tool.description = "Execute shell commands. Use this to run bash commands, scripts, or interact with the system."
        self.tools.append(shell_tool)
        
        logger.info("✓ Added Python REPL and Bash tools")
        
        # Create a persistent event loop for MCP operations
        self.mcp_loop = asyncio.new_event_loop()
        self.mcp_thread = threading.Thread(
            target=self.mcp_loop.run_forever,
            daemon=True,
            name="MCP-EventLoop"
        )
        self.mcp_thread.start()
        logger.info("Started MCP event loop thread")
        
        # Initialize MCP tools
        if self.mcp_servers:
            try:
                # Initialize MCP in the dedicated loop
                future = asyncio.run_coroutine_threadsafe(
                    self._init_mcp_async(),
                    self.mcp_loop
                )
                mcp_tools, self.cleanup_func = future.result(timeout=30)
                
                if mcp_tools:
                    # Process and wrap each MCP tool
                    for tool in mcp_tools:
                        # Extract tool metadata for parameter introspection
                        self._extract_tool_metadata(tool)
                        
                        # Wrap the tool with enhanced handling
                        wrapped_tool = self._wrap_mcp_tool_enhanced(tool)
                        self.tools.append(wrapped_tool)
                    
                    logger.info(f"✓ Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")
                    
                    # Log tool details
                    for name, info in self.mcp_tools_info.items():
                        logger.info(f"  - {name}: {info.get('params', 'No params')}")
                    
            except Exception as e:
                logger.error(f"MCP initialization failed: {e}")
                logger.info("Continuing without MCP tools...")
    
    def _extract_tool_metadata(self, mcp_tool):
        """Extract and store metadata about MCP tool parameters"""
        tool_info = {
            "name": mcp_tool.name,
            "description": mcp_tool.description,
            "params": {}
        }
        
        # Try to extract parameter information from the tool
        if hasattr(mcp_tool, 'args_schema'):
            schema = mcp_tool.args_schema
            if hasattr(schema, 'schema'):
                schema_dict = schema.schema()
                properties = schema_dict.get('properties', {})
                required = schema_dict.get('required', [])
                
                for param_name, param_info in properties.items():
                    tool_info["params"][param_name] = {
                        "type": param_info.get('type', 'string'),
                        "description": param_info.get('description', ''),
                        "required": param_name in required,
                        "default": param_info.get('default', None)
                    }
        
        # If no schema, try to parse from description
        elif 'parameter' in mcp_tool.description.lower():
            # Simple heuristic to extract parameter names from description
            desc_lower = mcp_tool.description.lower()
            if 'text' in desc_lower:
                tool_info["params"]["text"] = {
                    "type": "string",
                    "description": "Text input",
                    "required": True
                }
            elif 'message' in desc_lower:
                tool_info["params"]["message"] = {
                    "type": "string",
                    "description": "Message input",
                    "required": True
                }
            elif 'input' in desc_lower:
                tool_info["params"]["input"] = {
                    "type": "string",
                    "description": "Input value",
                    "required": True
                }
        
        self.mcp_tools_info[mcp_tool.name] = tool_info
        logger.info(f"Extracted metadata for {mcp_tool.name}: {tool_info['params']}")
    
    def _wrap_mcp_tool_enhanced(self, mcp_tool):
        """Enhanced wrapper for MCP tools with better parameter handling"""
        from langchain.tools import Tool
        
        tool_metadata = self.mcp_tools_info.get(mcp_tool.name, {})
        
        def sync_wrapper(*args, **kwargs):
            """Enhanced synchronous wrapper with parameter introspection"""
            try:
                # Log incoming arguments
                logger.info(f"MCP tool '{mcp_tool.name}' called with args={args}, kwargs={kwargs}")
                
                # Build proper kwargs based on tool metadata
                final_kwargs = {}
                
                # If we have metadata about parameters
                if tool_metadata.get("params"):
                    param_info = tool_metadata["params"]
                    
                    # Handle positional arguments
                    if args and not kwargs:
                        if len(args) == 1:
                            arg = args[0]
                            
                            # If it's already a dict with the right keys, use it
                            if isinstance(arg, dict):
                                # Check if dict has the expected parameters
                                for param_name in param_info:
                                    if param_name in arg:
                                        final_kwargs[param_name] = arg[param_name]
                                
                                # If no matching params found, treat the whole dict as input
                                if not final_kwargs:
                                    # Find the first required string parameter
                                    for param_name, info in param_info.items():
                                        if info.get("required") and info.get("type") == "string":
                                            final_kwargs[param_name] = str(arg)
                                            break
                            
                            # If it's a string, assign to the first string parameter
                            elif isinstance(arg, str):
                                for param_name, info in param_info.items():
                                    if info.get("type") == "string":
                                        final_kwargs[param_name] = arg
                                        break
                    
                    # Override with any explicit kwargs
                    final_kwargs.update(kwargs)
                
                else:
                    # No metadata, use fallback logic
                    if args and not kwargs:
                        if len(args) == 1:
                            if isinstance(args[0], dict):
                                final_kwargs = args[0]
                            else:
                                # Try common parameter names
                                final_kwargs = {"text": str(args[0])}
                    else:
                        final_kwargs = kwargs
                
                # Ensure all required parameters are present
                missing_params = []
                if tool_metadata.get("params"):
                    for param_name, info in tool_metadata["params"].items():
                        if info.get("required") and param_name not in final_kwargs:
                            # Try to provide a default or raise an error
                            if info.get("default") is not None:
                                final_kwargs[param_name] = info["default"]
                            else:
                                missing_params.append(param_name)
                
                if missing_params:
                    error_msg = f"Missing required parameters for '{mcp_tool.name}': {', '.join(missing_params)}"
                    logger.error(error_msg)
                    expected_format = {p: f"<{info['type']}>" for p, info in tool_metadata["params"].items() if info.get("required")}
                    # Use repr to safely format the JSON without template issues
                    return f"Error: {error_msg}\nExpected format: " + repr(expected_format)
                
                # Log the final call
                logger.info(f"Calling MCP tool '{mcp_tool.name}' with final kwargs: {final_kwargs}")
                
                # Submit the async operation to the MCP event loop
                future = asyncio.run_coroutine_threadsafe(
                    mcp_tool.ainvoke(final_kwargs),
                    self.mcp_loop
                )
                
                # Wait for the result
                result = future.result(timeout=30)
                
                logger.info(f"MCP tool '{mcp_tool.name}' returned: {result}")
                
                # Format the result
                if isinstance(result, dict):
                    return json.dumps(result, indent=2)
                elif isinstance(result, (list, tuple)):
                    return json.dumps(result, indent=2)
                else:
                    return str(result)
                    
            except asyncio.TimeoutError:
                error_msg = f"MCP tool '{mcp_tool.name}' timed out"
                logger.error(error_msg)
                return error_msg
            except Exception as e:
                error_msg = f"MCP tool '{mcp_tool.name}' error: {str(e)}"
                logger.error(error_msg)
                import traceback
                traceback.print_exc()
                return error_msg
        
        # Create enhanced description with parameter info
        enhanced_description = mcp_tool.description
        if tool_metadata.get("params"):
            param_desc = []
            required_params = []
            optional_params = []
            
            for param_name, info in tool_metadata["params"].items():
                desc_str = f"{param_name} ({info['type']})"
                if info.get('description'):
                    desc_str += f": {info['description']}"
                
                if info.get("required"):
                    required_params.append(desc_str)
                else:
                    optional_params.append(desc_str)
            
            if required_params:
                enhanced_description += f"\nRequired parameters: {', '.join(required_params)}"
            if optional_params:
                enhanced_description += f"\nOptional parameters: {', '.join(optional_params)}"
        
        # Create a new Tool with sync execution
        return Tool(
            name=mcp_tool.name,
            description=enhanced_description,
            func=sync_wrapper,
            return_direct=False
        )
    
    async def _init_mcp_async(self):
        """Initialize MCP tools asynchronously"""
        tools, cleanup = await convert_mcp_to_langchain_tools(self.mcp_servers)
        return tools, cleanup
    
    def _create_agents(self):
        """Create agents with proper tool handling"""
        for config in self.agent_configs:
            name = config["name"]
            model = config.get("model", "gpt-4o")
            temperature = config.get("temperature", 0.7)
            
            # Create LLM
            llm = ChatOpenAI(
                model=model,
                temperature=temperature,
                streaming=False,  # Disable streaming for better tool handling
                request_timeout=60
            )
            
            # Select tools based on agent type
            if name == "Python Coder":
                # Python coder gets Python REPL and Bash tools
                agent_tools = [t for t in self.tools if t.name in ["python_repl", "bash_command"]]
            elif name == "Data Analyst":
                # Data analyst gets all tools
                agent_tools = self.tools
            else:
                # General assistant gets MCP tools but not code execution
                agent_tools = [t for t in self.tools if t.name not in ["python_repl", "bash_command"]]
            
            # Create prompt with enhanced instructions
            system_message = self._get_enhanced_system_message(name, agent_tools)
            prompt = ChatPromptTemplate.from_messages([
                ("system", system_message),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad")
            ])
            
            # Create agent
            agent = create_tool_calling_agent(llm, agent_tools, prompt)
            
            # Create executor with proper async handling
            executor = AgentExecutor(
                agent=agent,
                tools=agent_tools,
                verbose=True,  # Enable verbose for debugging
                return_intermediate_steps=True,
                max_iterations=self.max_iterations,  # Use configurable iterations
                early_stopping_method="generate",
                handle_parsing_errors=True
            )
            
            self.executors[name] = executor
            self.agents[name] = {
                "executor": executor,
                "tools": agent_tools,
                "config": config
            }
            
            logger.info(f"✓ Created agent: {name} with {len(agent_tools)} tools: {[t.name for t in agent_tools]}")
    
    def _get_enhanced_system_message(self, agent_name: str, tools: List) -> str:
        """Get enhanced system message with detailed tool instructions"""
        tool_descriptions = []
        
        for tool in tools:
            desc = f"- {tool.name}: {tool.description}"
            
            # Add parameter details for MCP tools
            if tool.name in self.mcp_tools_info:
                params = self.mcp_tools_info[tool.name].get("params", {})
                if params:
                    param_list = []
                    for param_name, info in params.items():
                        req = "required" if info.get("required") else "optional"
                        param_list.append(f"{param_name} ({info['type']}, {req})")
                    desc += f"\n  Parameters: {', '.join(param_list)}"
            
            tool_descriptions.append(desc)
        
        tools_text = "\n".join(tool_descriptions)
        
        if agent_name == "Python Coder":
            message = "You are a Python programming assistant with code execution capabilities.\n\n"
            message += "Available tools:\n"
            message += tools_text + "\n\n"
            message += "Python REPL Instructions:\n"
            message += "- The python_repl tool executes Python code and returns the output\n"
            message += "- To see results, you MUST use print() statements or put expressions as the last line\n"
            message += "- Variables persist between executions in the same session\n"
            message += "- Always include print statements for intermediate results\n"
            message += "- For data analysis or visualizations, always explicitly print results\n\n"
            message += "General Instructions:\n"
            message += "- Write clean, well-commented Python code\n"
            message += "- Always execute code using the python_repl tool to show results\n"
            message += "- Use bash_command for system operations like installing packages\n"
            message += "- Explain your code and results clearly"
            return message
        
        elif agent_name == "Data Analyst":
            message = "You are a data analyst assistant with access to various analysis tools.\n\n"
            message += "Available tools:\n"
            message += tools_text + "\n\n"
            message += "Python REPL Instructions:\n"
            message += "- ALWAYS use print() to display results, dataframes, statistics, etc.\n"
            message += "- For pandas DataFrames: use print(df), print(df.head()), print(df.describe())\n"
            message += "- For plots: use plt.show() after creating visualizations\n"
            message += "- Variables persist between executions\n\n"
            message += "General Instructions:\n"
            message += "- Always use the appropriate tool when asked\n"
            message += "- For tools with parameters, use the exact parameter names as shown\n"
            message += "- Combine multiple tools when needed for complex analysis\n"
            message += "- Provide clear explanations of your findings\n"
            message += "- Use python_repl for data processing and visualization\n"
            message += "- Use bash_command for file operations"
            return message
        
        else:
            message = "You are a helpful " + agent_name + " with access to various tools.\n\n"
            message += "Available tools:\n"
            message += tools_text + "\n\n"
            message += "Instructions:\n"
            message += "- Use tools whenever they can help answer questions\n"
            message += "- For tools with parameters, use the exact parameter names as specified\n"
            message += "- Pay close attention to whether a parameter is required or optional\n"
            message += "- Provide clear and helpful responses\n"
            message += "- Explain what tools you're using and why"
            return message
    
    def process_message(self, audio_input, text_input, agent_name, history, tts_enabled):
        """Process user message"""
        try:
            # Get input
            if audio_input is not None:
                input_text = self.audio.transcribe(audio_input)
            else:
                input_text = text_input
            
            if not input_text.strip():
                return None, history, "", "No input detected"
            
            logger.info(f"Processing: '{input_text[:50]}...' with {agent_name}")
            
            # Get response from agent
            response = self._run_agent(agent_name, input_text, history)
            
            # Generate audio if enabled
            audio_response = None
            if self.audio.has_tts and tts_enabled and len(response) < 1000:
                audio_response = self.audio.speak(response[:500])
            
            # Update history
            new_history = history + [
                {"role": "user", "content": input_text},
                {"role": "assistant", "content": response}
            ]
            
            return audio_response, new_history, "", input_text
            
        except Exception as e:
            logger.error(f"Processing error: {e}")
            import traceback
            traceback.print_exc()
            return None, history, "", f"Error: {str(e)}"
    
    def _run_agent(self, agent_name: str, message: str, history: List) -> str:
        """Run agent with proper async handling"""
        if agent_name not in self.agents:
            return f"Agent '{agent_name}' not found"
        
        executor = self.agents[agent_name]["executor"]
        
        # Build chat history
        chat_history = []
        for h in history[-10:]:  # Last 10 messages
            if h["role"] == "user":
                chat_history.append(HumanMessage(content=h["content"]))
            else:
                chat_history.append(AIMessage(content=h["content"]))
        
        try:
            # Use sync invoke to avoid event loop issues
            logger.info(f"Invoking agent executor for: {message[:50]}...")
            
            result = executor.invoke({
                "input": message,
                "chat_history": chat_history
            })
            
            logger.info(f"Agent executor returned: {type(result)}")
            
            # Extract response
            if isinstance(result, dict):
                # Log intermediate steps if available
                if "intermediate_steps" in result:
                    for step in result["intermediate_steps"]:
                        if len(step) >= 2:
                            action, observation = step[0], step[1]
                            logger.info(f"Tool used: {action.tool if hasattr(action, 'tool') else 'unknown'}")
                            logger.info(f"Tool result: {str(observation)[:200]}...")
                
                # Return the output
                return result.get("output", "No response generated")
            else:
                return str(result)
                
        except Exception as e:
            logger.error(f"Agent execution error: {e}", exc_info=True)
            return f"Error executing agent: {str(e)}"
    
    def create_interface(self):
        """Create Gradio interface"""
        with gr.Blocks(
            title="Enhanced Multi-Agent Voice Assistant",
            theme=gr.themes.Soft()
        ) as interface:
            gr.Markdown("# 🤖 Enhanced Multi-Agent Voice Assistant with MCP Tools")
            gr.Markdown("Features: Python REPL, Bash commands, and MCP tool integration")
            
            with gr.Row():
                with gr.Column(scale=1):
                    agent_dropdown = gr.Dropdown(
                        choices=[c["name"] for c in self.agent_configs],
                        value=self.agent_configs[0]["name"],
                        label="Select Agent"
                    )
                    
                    max_iterations_slider = gr.Slider(
                        minimum=1,
                        maximum=10,
                        value=self.max_iterations,
                        step=1,
                        label="Max Iterations",
                        info="Maximum number of tool calls the agent can make"
                    )
                    
                    agent_info = gr.Markdown("")
                    
                    if self.audio.has_stt:
                        audio_input = gr.Audio(
                            sources=["microphone"],
                            type="numpy",
                            label="🎤 Voice Input"
                        )
                    else:
                        audio_input = None
                    
                    text_input = gr.Textbox(
                        label="Text Input",
                        placeholder="Type your message...",
                        lines=3
                    )
                    
                    send_btn = gr.Button("Send", variant="primary")
                    
                    if self.audio.has_tts:
                        tts_toggle = gr.Checkbox(
                            label="Enable Text-to-Speech",
                            value=True
                        )
                        audio_output = gr.Audio(
                            label="Voice Response",
                            type="numpy",
                            autoplay=True
                        )
                    else:
                        tts_toggle = gr.State(False)
                        audio_output = None
                
                with gr.Column(scale=2):
                    chatbot = gr.Chatbot(
                        label="Conversation",
                        height=600
                    )
                    
                    last_input = gr.Textbox(
                        label="Last Input",
                        interactive=False
                    )
                    
                    clear_btn = gr.Button("Clear Chat")
            
            # State
            history_state = gr.State([])
            
            # Update max iterations when slider changes
            def update_max_iterations(value):
                self.max_iterations = value
                # Update all existing executors
                for agent_name, agent_data in self.agents.items():
                    agent_data["executor"].max_iterations = value
                return f"Max iterations set to {value}"
            
            max_iterations_slider.change(
                update_max_iterations,
                inputs=[max_iterations_slider],
                outputs=[gr.State()]  # Just update internally
            )
            
            # Update agent info
            def show_agent_info(name):
                agent = self.agents.get(name, {})
                tools = agent.get("tools", [])
                tool_info = []
                
                for t in tools:
                    info = f"**{t.name}**"
                    if t.name in self.mcp_tools_info:
                        params = self.mcp_tools_info[t.name].get("params", {})
                        if params:
                            param_list = [f"{p} ({info['type']})" for p, info in params.items()]
                            info += f" - Params: {', '.join(param_list)}"
                    tool_info.append(info)
                
                return f"**{name}**\n\nAvailable Tools:\n" + "\n".join(tool_info)
            
            agent_dropdown.change(
                show_agent_info,
                inputs=[agent_dropdown],
                outputs=[agent_info]
            )
            
            # Process message
            send_btn.click(
                fn=self.process_message,
                inputs=[
                    audio_input if audio_input else gr.State(None),
                    text_input,
                    agent_dropdown,
                    history_state,
                    tts_toggle
                ],
                outputs=[
                    audio_output if audio_output else gr.State(None),
                    history_state,
                    text_input,
                    last_input
                ]
            )
            
            # Update chat
            history_state.change(
                fn=lambda h: [(m["content"], None) if m["role"] == "user" 
                             else (None, m["content"]) for m in h],
                inputs=[history_state],
                outputs=[chatbot]
            )
            
            # Clear
            clear_btn.click(
                fn=lambda: ([], [], "", ""),
                outputs=[chatbot, history_state, text_input, last_input]
            )
            
            # Initialize
            interface.load(
                fn=lambda: show_agent_info(self.agent_configs[0]["name"]),
                outputs=[agent_info]
            )
        
        return interface
    
    def _cleanup(self):
        """Cleanup resources"""
        logger.info("Cleaning up...")
        
        # Cleanup MCP
        if self.cleanup_func and self.mcp_loop:
            try:
                future = asyncio.run_coroutine_threadsafe(
                    self.cleanup_func(),
                    self.mcp_loop
                )
                future.result(timeout=5)
            except Exception as e:
                logger.error(f"Cleanup error: {e}")
        
        # Stop MCP event loop
        if self.mcp_loop:
            self.mcp_loop.call_soon_threadsafe(self.mcp_loop.stop)
            if self.mcp_thread:
                self.mcp_thread.join(timeout=2)

# Usage
if __name__ == "__main__":
    MCP_SERVERS = {
        "local_http": {"url": "http://127.0.0.1:8000/mcp"}
    }
    
    AGENT_CONFIGS = [
        {
            "name": "Data Analyst",
            "model": "gpt-4o",
            "temperature": 0.3,
            "description": "Data analysis with all available tools"
        },
        {
            "name": "Python Coder",
            "model": "gpt-4o",
            "temperature": 0.2,
            "description": "Python programming with code execution"
        },
        {
            "name": "Assistant",
            "model": "gpt-4o",
            "temperature": 0.7,
            "description": "General assistant with MCP tools"
        }
    ]
    
    # You can set max_iterations here (default is 5)
    dashboard = EnhancedMCPDashboard(MCP_SERVERS, AGENT_CONFIGS, max_iterations=5)
    interface = dashboard.create_interface()
    interface.launch(share=True)

INFO:__main__:Initializing enhanced dashboard...
INFO:__main__:✓ Speech recognition available
INFO:__main__:✓ Text-to-speech available
INFO:__main__:Starting system initialization...
INFO:__main__:✓ Added Python REPL and Bash tools
INFO:__main__:Started MCP event loop thread
INFO:langchain_mcp_tools.langchain_mcp_tools:MCP server "local_http": initializing with: {'url': 'http://127.0.0.1:8000/mcp'}
INFO:langchain_mcp_tools.langchain_mcp_tools:MCP server "local_http": Pre-validating authentication


ERROR:langchain_mcp_tools.langchain_mcp_tools:MCP server "local_http": error during initialization: MCP server "local_http": Connection failed: All connection attempts failed
ERROR:__main__:MCP initialization failed: MCP server "local_http": Connection failed: All connection attempts failed
INFO:__main__:Continuing without MCP tools...
INFO:__main__:✓ Created agent: Data Analyst with 2 tools: ['python_repl', 'bash_command']
INFO:__main__:✓ Created agent: Python Coder with 2 tools: ['python_repl', 'bash_command']
INFO:__main__:✓ Created agent: Assistant with 0 tools: []
INFO:__main__:✓ System initialization complete
/tmp/ipykernel_26549/54127133.py:730: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e49987cddd686d3b71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


INFO:__main__:Processing: 'hi
...' with Data Analyst
INFO:__main__:Invoking agent executor for: hi
...




> Entering new AgentExecutor chain...


...

INFO:__main__:Processing: 'can you test the audio...' with Data Analyst
INFO:__main__:Invoking agent executor for: can you test the audio...




> Entering new AgentExecutor chain...


INFO:__main__:Agent executor returned: <class 'dict'>


I don't have the capability to test audio directly. However, I can guide you through the process of testing audio on your device. Let me know what you need help with!

> Finished chain.


ERROR:__main__:TTS error: [Errno 2] No such file or directory: 'ffprobe'
